# RSM8413: Case 1

#### Team 15: Joanne Chien, Harsh Pinge, Venkata Surya Sai Nikhil Garimella, Sharon Xiao, Jitao Zhang

In [32]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from numpy import arange
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import make_column_transformer

## 1. Cleaning the dataset

### 1.1 First glimpse of the dataset & Initial Cleanings (for EDA)

In [33]:
ebay_raw = pd.read_excel("data/ebayAuctions.xlsx", sheet_name = "eBay auctions")
ebay_raw.head()

,Category,Currency,sellerRating,Duration,endDay,ClosePrice,OpenPrice,Competitive?
0,Music/Movie/Game,US,3249,5,Mon,0.01,0.01,0
1,Music/Movie/Game,US,3249,5,Mon,0.01,0.01,0
2,Music/Movie/Game,US,3249,5,Mon,0.01,0.01,0
3,Music/Movie/Game,US,3249,5,Mon,0.01,0.01,0
4,Music/Movie/Game,US,3249,5,Mon,0.01,0.01,0


In [34]:
ebay_raw.shape

(1972, 8)

In [35]:
ebay_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1972 entries, 0 to 1971
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Category      1972 non-null   object 
 1   Currency      1972 non-null   object 
 2   sellerRating  1972 non-null   int64  
 3   Duration      1972 non-null   int64  
 4   endDay        1972 non-null   object 
 5   ClosePrice    1972 non-null   float64
 6   OpenPrice     1972 non-null   float64
 7   Competitive?  1972 non-null   int64  
dtypes: float64(2), int64(3), object(3)
memory usage: 123.4+ KB


In [36]:
ebay_raw.describe()

,sellerRating,Duration,ClosePrice,OpenPrice,Competitive?
count,1972.000000,1972.000000,1972.000000,1972.000000,1972.000000
mean,3560.238337,6.486308,36.449087,12.926268,0.540568
std,5973.011033,2.052906,89.493858,38.856149,0.498478
min,0.000000,1.000000,0.010000,0.010000,0.000000
25%,595.000000,5.000000,4.907500,1.230000,0.000000
50%,1853.000000,7.000000,9.995000,4.500000,1.000000
75%,3380.000000,7.000000,28.000000,9.990000,1.000000
max,37727.000000,10.000000,999.000000,999.000000,1.000000


In [37]:
ebay_raw['Category'].value_counts()

Category
Music/Movie/Game        403
Collectibles            239
Toys/Hobbies            234
Automotive              178
Antique/Art/Craft       177
SportingGoods           124
Clothing/Accessories    119
Home/Garden             102
Jewelry                  82
Health/Beauty            64
Electronics              55
Books                    54
Coins/Stamps             37
Computer                 36
Pottery/Glass            20
Business/Industrial      18
EverythingElse           17
Photography              13
Name: count, dtype: int64

In [38]:
ebay_raw['Currency'].value_counts()

Currency
US     1292
EUR     533
GBP     147
Name: count, dtype: int64

In [39]:
ebay_raw['endDay'].value_counts()

endDay
Mon    548
Sat    351
Sun    338
Fri    287
Thu    202
Tue    171
Wed     75
Name: count, dtype: int64

In [40]:
# Rename the response column
ebay = ebay_raw.rename(columns={'Competitive?': 'competitive'})
ebay.head()

,Category,Currency,sellerRating,Duration,endDay,ClosePrice,OpenPrice,competitive
0,Music/Movie/Game,US,3249,5,Mon,0.01,0.01,0
1,Music/Movie/Game,US,3249,5,Mon,0.01,0.01,0
2,Music/Movie/Game,US,3249,5,Mon,0.01,0.01,0
3,Music/Movie/Game,US,3249,5,Mon,0.01,0.01,0
4,Music/Movie/Game,US,3249,5,Mon,0.01,0.01,0


In [41]:
# convert all prices into USD (preparation)
# https://www.xe.com/currencytables/?from=GBP&date=2004-12-31
# FX on May 31, 2004
eur_to_usd = 1.2186
gbp_to_usd = 1.8316

def convert_fx(row):
    if row['Currency'] == 'EUR':
        row['ClosePrice'] *= eur_to_usd
        row['OpenPrice'] *= eur_to_usd
    elif row['Currency'] == 'GBP':
        row['ClosePrice'] *= gbp_to_usd
        row['OpenPrice'] *= gbp_to_usd
    return row

In [42]:
# convert all prices into USD (continue) & get rid of the currency column
ebay = ebay.apply(convert_fx, axis = 1)
ebay['ClosePrice'] = ebay['ClosePrice'].round(2)
ebay['OpenPrice'] = ebay['OpenPrice'].round(2)

ebay = ebay.drop(columns=['Currency'])

In [43]:
ebay.tail(10)

,Category,sellerRating,Duration,endDay,ClosePrice,OpenPrice,competitive
1962,Automotive,0,10,Sun,590.00,99.00,1
1963,Automotive,4,10,Sun,951.59,149.85,1
1964,SportingGoods,1,7,Fri,417.99,200.00,1
1965,Automotive,97,3,Fri,290.00,290.00,0
1966,Automotive,142,7,Sat,521.55,200.00,1
1967,Automotive,2992,5,Sun,359.95,359.95,0
1968,Automotive,21,5,Sat,610.00,300.00,1
1969,Automotive,1400,5,Mon,549.00,549.00,0
1970,Automotive,57,7,Fri,820.00,650.00,1
1971,Automotive,145,7,Sat,999.00,999.00,0


### 1.2 Building the preprocessor (for fitting the models)

In [44]:
numeric_features = ["sellerRating",
                    "Duration",
                    "ClosePrice",
                    "OpenPrice"]
categorical_features = ["Category",
                        "endDay"]


In [45]:
numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(sparse_output = False)

In [46]:
preprocessor = make_column_transformer(
    (numeric_transformer, numeric_features),
    (categorical_transformer, categorical_features)
)

preprocessor

,transformers,"[('standardscaler', ...), ('onehotencoder', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True
,force_int_remainder_cols,'deprecated'
,copy,True
,with_mean,True
,with_std,True
